# 1. 신입 데이터 분석가를 위한 실전 미션!

< 주어지는 데이터 >
1. 고객 정보 (직접 생성)
    - customer_id : 101, 102, 103, 104, 105, 106
    - name : 'Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank'
    - age : 25,32,45,38,29,41
2. 구매 내역
    - order_history 데이터셋으로 가져오기

< 요구 사항 >
1. 구매 데이터의 price 컬럼에 있는 결측치는 전체 상품의 "평균 가격"으로 채우시오.
2. price가 1000을 초과하는 데이터는 이상치로 간주하고, 전체 상품의 "중앙값"으로 대체하시오.
3. 정제된 두 데이터프레임을 customer_id 기준으로 병합하시오.
4. 최종 병합된 데이터프레임에 대한 기술 통계량을 도출하시오.

In [27]:
import numpy as np
from pandas import DataFrame
from hossam import load_data
from pandas import merge
from sklearn.impute import SimpleImputer
from pandas import CategoricalDtype

In [28]:
customer_info={'customer_id':[101,102,103,104,105,106],'name':['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank'],'age':[25,32,45,38,29,41]}
df=DataFrame(customer_info)
df

,customer_id,name,age
0,101,Alice,25
1,102,Bob,32
2,103,Charlie,45
3,104,David,38
4,105,Eve,29
5,106,Frank,41


In [29]:
origin=load_data('order_history')
origin

[data] https://data.hossam.kr/data/lab04/order_history.xlsx
[desc] 신입 데이터 분석가를 위한 실전 미션!

field        description
-----------  -------------
purchase_id  일련번호
customer_id  고객번호
item         상품명
price        금액



,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.0
2,102,Mouse,25.0
3,103,Keyboard,NaN
4,101,Monitor,300.0
5,104,Webcam,5000.0
6,102,USB Hub,30.0
7,105,Laptop,1100.0
8,103,Mouse,NaN
9,101,External HDD,80.0


In [32]:
imr=SimpleImputer(missing_values=np.nan, strategy='mean')
df_reshape=origin['price'].values.reshape(-1,1)
df_imr=imr.fit_transform(df_reshape)
hello_df=origin.copy()
hello_df['price']=df_imr
hello_df

,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,5000.000
6,102,USB Hub,30.000
7,105,Laptop,1100.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [5]:
hello_df.loc[hello_df['price']>1000,'price']=np.nan
imr=SimpleImputer(missing_values=np.nan, strategy='median')
df_reshape_1=hello_df['price'].values.reshape(-1,1)
df_imr=imr.fit_transform(df_reshape_1)
my_df=hello_df.copy()
my_df['price']=df_imr
my_df

,customer_id,item,price
purchase_id,,,
1,101,Laptop,150.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,150.000
6,102,USB Hub,30.000
7,105,Laptop,150.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [6]:
df_final=merge(df, my_df, on='customer_id')
df_final

,customer_id,name,age,item,price
0,101,Alice,25,Laptop,150.000
1,101,Alice,25,Monitor,300.000
2,101,Alice,25,External HDD,80.000
3,102,Bob,32,Mouse,25.000
4,102,Bob,32,USB Hub,30.000
5,103,Charlie,45,Keyboard,985.625
6,103,Charlie,45,Mouse,985.625
7,104,David,38,Webcam,150.000
8,105,Eve,29,Laptop,150.000
9,106,Frank,41,Docking Station,150.000


In [7]:
df_final_final=df_final.astype({'customer_id':'category'})
df_final_final.describe()

,age,price
count,10.00000,10.000000
mean,33.70000,300.625000
std,8.04225,369.246135
min,25.00000,25.000000
25%,26.00000,97.500000
50%,32.00000,150.000000
75%,40.25000,262.500000
max,45.00000,985.625000


# 2. 흩어진 판매 데이터 통합하기

판매 데이터 "shop_강남, shop_종로"를 불러와서 아래 작업을 수행하시오

1. 강남점 데이터에는 Mouse 상품의 가격(Price)이 누락되어 있습니다. 이 값을 8000으로 채우시오.
2. 종로점 데이터에는 Keyboard 상품의 주문 수량(Quantity)이 999로 잘못 입력되어 있습니다. 실제 수량은 9이므로, 이 값을 수정하시오.
3. 두 개의 DataFrame을 하나의 DataFrame으로 합치시오. 합치는 과정에서 기존의 인덱스는 의미가 없으므로, 새로운 인덱스를 부여하세요.
4. 화면에 최종 DataFrame의 전체 내용을 출력하시오.

In [8]:
from hossam import load_data
from pandas import merge
from pandas import concat
import numpy as np

In [9]:
강남df=load_data('shop_강남')
강남df

[data] https://data.hossam.kr/data/lab04/shop_강남.xlsx
[desc] 흩어진 판매 데이터 통합하기(인덱스/메타데이터 없음)
[!] Cannot read metadata


,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,NaN
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0


In [10]:
종로df=load_data('shop_종로')
종로df

[data] https://data.hossam.kr/data/lab04/shop_종로.xlsx
[desc] 흩어진 판매 데이터 통합하기(인덱스/메타데이터 없음)
[!] Cannot read metadata


,OrderID,Product,Quantity,Price
0,B001,Mouse,15,8000
1,B002,Keyboard,999,14500
2,B003,Webcam,10,30000
3,B004,Mouse,18,8000


In [11]:
강남df.loc[강남df['Product']=='Mouse','Price']=8000
강남df

,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,8000.0
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0


In [12]:
종로df.loc[종로df['Product']=='Keyboard','Quantity']=9
종로df

,OrderID,Product,Quantity,Price
0,B001,Mouse,15,8000
1,B002,Keyboard,9,14500
2,B003,Webcam,10,30000
3,B004,Mouse,18,8000


In [13]:
final_df=concat([강남df,종로df],ignore_index=True)
final_df

,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,8000.0
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0
4,B001,Mouse,15,8000.0
5,B002,Keyboard,9,14500.0
6,B003,Webcam,10,30000.0
7,B004,Mouse,18,8000.0


# 3. 마케팅팀 신입사원의 첫 번째 미션

vip_customer 데이터셋을 불러와서 아래의 데이터를 찾아내시오.

1. 아래의 조건들을 모두 만족하는 VIP 고객을 찾아내어, 그들의 명단을 나타내시오.
    - 우수 고객 : 총 구매액(TotalSpending)이 100,000원 이상인 고객
    - 활동적인 고객 : 서울(City)에 거주하는 고객
2. 위 조건을 모두 만족하는 고객들을 총 구매액(TotalSpending)이 높은 순서대로 정렬하여, 상위 5명의 고객 정보를 추출하시오.
3. 추출된 5명의 고객 데이터에서 Name, City, TotalSpending 세 개의 열만 선택하여 최종 명단을 만드시오.

In [14]:
from hossam import load_data

In [15]:
origin=load_data('vip_customer')
origin

[data] https://data.hossam.kr/data/lab04/vip_customer.xlsx
[desc] VIP 고객 목록 데이터

field             description
----------------  ----------------
CustomerID        고객번호
Name              고객명
Age               나이
City              거주지역
JoinDate          가입일
TotalSpending     총 지출액
LastPurchaseDate  마지막 구매 날짜



,Name,Age,City,JoinDate,TotalSpending,LastPurchaseDate
CustomerID,,,,,,
1001,김철수,35,서울,2022-01-15,150000,2023-11-01
1002,이영희,28,부산,2021-03-22,80000,2023-10-15
1003,박민준,42,서울,2022-07-10,250000,2023-11-10
1004,최지우,25,인천,2023-02-20,50000,2023-09-25
1005,정다솜,31,서울,2021-11-05,320000,2023-11-12
1006,윤현우,38,부산,2022-05-30,120000,2023-11-05
1007,강예진,29,서울,2023-01-15,85000,2023-10-22
1008,오승민,45,광주,2021-08-12,180000,2023-11-08
1009,한지민,27,서울,2022-10-01,450000,2023-11-15


In [16]:
# 명단 탐색
real_vip=origin.query('TotalSpending>=100000 and City=="서울"')
real_vip

,Name,Age,City,JoinDate,TotalSpending,LastPurchaseDate
CustomerID,,,,,,
1001,김철수,35,서울,2022-01-15,150000,2023-11-01
1003,박민준,42,서울,2022-07-10,250000,2023-11-10
1005,정다솜,31,서울,2021-11-05,320000,2023-11-12
1009,한지민,27,서울,2022-10-01,450000,2023-11-15
1011,임도윤,39,서울,2021-06-18,210000,2023-11-02
1013,황재민,41,서울,2022-09-05,190000,2023-11-07
1015,고은별,36,서울,2023-05-25,130000,2023-11-11


In [17]:
# 정렬
my_df=real_vip.sort_values('TotalSpending',ascending=False)
my_df.head()

,Name,Age,City,JoinDate,TotalSpending,LastPurchaseDate
CustomerID,,,,,,
1009,한지민,27,서울,2022-10-01,450000,2023-11-15
1005,정다솜,31,서울,2021-11-05,320000,2023-11-12
1003,박민준,42,서울,2022-07-10,250000,2023-11-10
1011,임도윤,39,서울,2021-06-18,210000,2023-11-02
1013,황재민,41,서울,2022-09-05,190000,2023-11-07


In [18]:
# 최종 명단
final_df=my_df.head().filter(['Name','City','TotalSpending'])
final_df

,Name,City,TotalSpending
CustomerID,,,
1009,한지민,서울,450000
1005,정다솜,서울,320000
1003,박민준,서울,250000
1011,임도윤,서울,210000
1013,황재민,서울,190000


# 4. 온라인 쇼핑몰의 다음 분기 프로모션 전략 수립

데이터셋 shop_order_list를 가져와서 아래 내용을 수행하시오.

1. 데이터 정제
    - 데이터에 결측치가 있는지 확인하고, 'Price' 컬럼에 있는 결측치는 전체 상품의 평균 'Price'로 대체하시오.
2. 베스트셀러 카테고리 분석
    - 'Category' 별로 주문건수를 계산하여 가장 많은 주문이 발생한 카테고리 상위 3개를 찾으시오.
3. 지역별 구매력 분석
    - 'Region'별 평균 'Price'를 계산하여 어느 지역의 고객들이 평균적으로 가장 비싼 상품을 구매하는지 확인하시오.
    - 결과는 평균 구매액이 높은 순서대로 정렬하시오.

In [19]:
from hossam import load_data
import numpy as np

In [20]:
origin=load_data('shop_order_list')
origin

[data] https://data.hossam.kr/data/lab04/shop_order_list.xlsx
[desc] 온라인 쇼핑몰의 최근 고객 주문 데이터
[!] Cannot read metadata


,UserID,Product,Price,Category,Region,OrderDate
OrderID,,,,,,
1001,user001,Laptop,1200.0,Electronics,Seoul,2023-01-15
1002,user002,Mouse,25.0,Electronics,Busan,2023-01-16
1003,user003,Keyboard,70.0,Electronics,Seoul,2023-01-17
1004,user001,Book,15.0,Books,Incheon,2023-01-18
1005,user004,T-shirt,30.0,Apparel,Gwangju,2023-01-19
1006,user002,Book,20.0,Books,Seoul,2023-01-20
1007,user005,Jeans,80.0,Apparel,Busan,2023-02-21
1008,user001,Monitor,300.0,Electronics,Seoul,2023-02-22
1009,user006,Book,18.0,Books,Daegu,2023-02-23


In [21]:
origin.isna().sum()

UserID       0
Product      0
Price        2
Category     0
Region       0
OrderDate    0
dtype: int64

In [22]:
price_mean=origin['Price'].mean()
my_df=origin.fillna(price_mean)
my_df

,UserID,Product,Price,Category,Region,OrderDate
OrderID,,,,,,
1001,user001,Laptop,1200.000000,Electronics,Seoul,2023-01-15
1002,user002,Mouse,25.000000,Electronics,Busan,2023-01-16
1003,user003,Keyboard,70.000000,Electronics,Seoul,2023-01-17
1004,user001,Book,15.000000,Books,Incheon,2023-01-18
1005,user004,T-shirt,30.000000,Apparel,Gwangju,2023-01-19
1006,user002,Book,20.000000,Books,Seoul,2023-01-20
1007,user005,Jeans,80.000000,Apparel,Busan,2023-02-21
1008,user001,Monitor,300.000000,Electronics,Seoul,2023-02-22
1009,user006,Book,18.000000,Books,Daegu,2023-02-23


In [23]:
sort_df=my_df.groupby(['Category']).count().sort_values(['UserID'],ascending=False)
print('가장 많은 주문이 발생한 카테고리 TOP 3:')
print(sort_df['UserID'].head(3))

가장 많은 주문이 발생한 카테고리 TOP 3:
Category
Electronics    6
Apparel        5
Books          5
Name: UserID, dtype: int64


In [24]:
# 정답
my_df['Category'].value_counts().head(3)

Category
Electronics    6
Books          5
Apparel        5
Name: count, dtype: int64

In [25]:
region_df=my_df.filter(['Region','Price']).groupby(['Region']).mean().sort_values('Price',ascending=False)
print('지역별 평균 구매액 (높은 순):')
print(region_df)

지역별 평균 구매액 (높은 순):
              Price
Region             
Seoul    367.259259
Busan     90.000000
Incheon   76.777778
Daegu     69.000000
Gwangju   16.500000
